In [8]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy.plotting import plot
import operator
from IPython.display import Image
from IPython.core.display import HTML 

In [9]:
def nullcline_intersection(sys, r):
    x, y = sp.symbols('x, y')
    eq1 = sys(x, y, r)[0]
    eq2 = sys(x, y, r)[1]
    sols = sp.solve([eq1, eq2], [x, y])
    return np.array(sols,dtype=float)

###### System of Equations

Full Commitment:
\begin{align}
\dot{k} &= k^\alpha - \left(\delta + n + x\right)k - c\\
\dot{c} &= c\left(\alpha k^{\alpha - 1} - \left(\delta + x\right) - \left(\rho + be^{-\gamma t}\right) \right)
\end{align}

No Commitment:
\begin{align}
\dot{k} &= k^\alpha - \left(\delta + n + x\right)k - c\\
\dot{c} &= c\left(\alpha k^{\alpha - 1} - \left(\delta + x + \lambda\right) \right)
\end{align}

###### Jacobian of the System

$$J(k,c)=
\begin{bmatrix}
\alpha k^{\alpha-1}-\left(\delta+n+x\right) & -1\\
\alpha(\alpha-1)ck^{\alpha-2} & \alpha k^{\alpha - 1} - \left(\delta + x\right) - \left(\rho + be^{-\gamma t}\right)\\
\end{bmatrix}$$


When the Jacobian is at a steady state the $$rho + be^{-\gamma t}$$ will be 0 as t approaches infinity.  

$$J(k,c)=
\begin{bmatrix}
\alpha k^{\alpha-1}-\left(\delta+n+x\right) & -1\\
\alpha(\alpha-1)ck^{\alpha-2} & 0\\
\end{bmatrix}$$


Through this we can find the eigenvalues and eigenvectors of the Jacobian matrix to check the stability of the system as  it is centered at the steady state. 

In [10]:
def jacsys(k,c,r):
    alpha, delta, xt, n, rho, b, gamma = r[0], r[1], r[2], r[3], r[4], r[5], r[6]
    J = np.array([[alpha*k**(alpha-1)-(delta+xt+n), -1],
    [c*alpha*(alpha-1)*k**(alpha-2), 0]])
    return J

In [11]:
# Use the RK4 solver to complete Linearizing the Jacobian 
def rk4(t,y,h,func,J):
    k1 = h*func(t,y,J)
    k2 = h*func(t+0.5*h,y+0.5*k1,J)
    k3 = h*func(t+0.5*h,y+0.5*k2,J)
    k4 = h*func(t+h,y+k3,J)
    return (k1 + 2*k2 + 2*k3 + k4)/6

# Homebrew Solver: Euler's Method
def rk4_solve(t0,tf,h,y0,func,J):
    tv, yv, = [], []
    t, y = t0, y0
    while t < tf:
        y += rk4(t,y,h,func,J)
        t += h
        yv.append(y.copy())
        tv.append(t)
    return np.array(tv), np.array(yv)

# Take the values close to the steady-point to see the stability around each point placed in J
def F(t,U,J):
    # System of Equations
    return np.array([ J[0,0]*U[0] + J[0,1]*U[1],
                      J[1,0]*U[0] + J[1,1]*U[1]])

In [14]:
alpha, delta, xt, n, rho, b, gamma = 0.75, 0.05, 0.02, 0.01, 0.02, 0.5, 0.5
r = (alpha, delta, xt, n, rho, b, gamma)
lam = .03653

# Equations of k*_0 and c*_0 for No Commitment
kzed = (alpha/(delta + lam + xt))**(1/(1-alpha)) 
czed = kzed**alpha - (delta + n + xt)*kzed 

# Equations of k*_inf and c*_inf for Full Commitment
kinf = (alpha/(delta + rho + xt))**(1/(1-alpha)) 
cinf = kinf**alpha - (delta + n + xt)*kinf

In [21]:
# To Test Full or No Comittment change below
# True = Full Commitment 
# False = No Commitment
CommitmentType = False

if CommitmentType:    
    # Jacobian of the Full Comittment Case
    def dFull(k,c,r):
        alpha, delta, xt, n, rho, b, gamma = r[0], r[1], r[2], r[3], r[4], r[5], r[6]
        G1 = k**alpha - (delta+xt+n)*k - c
        G2 = c*(alpha*k**(alpha-1) - (delta+xt) - rho) # b*exp(-gamma*t) -> 0 as t -> infty
        return [G1, G2]
    
    eq = nullcline_intersection(dFull,r)
    Jacobian = jacsys(kinf,cinf,r) 
    Jentries = np.array([(Jacobian[0,0],Jacobian[0,1],Jacobian[1,0],Jacobian[1,1])])    
    print("The Full Commitment Case:")
    
else:
    # Jacobian of the No Comittment Case
    def dNo(k,c,r):
        alpha, delta, xt, n, rho, b, gamma = r[0], r[1], r[2], r[3], r[4], r[5], r[6]
        G1 = k**alpha - (delta+xt+n)*k - c
        G2 = c*(alpha*k**(alpha-1) - (delta+xt) - lam)
        return [G1, G2]
    
    eq = nullcline_intersection(dNo,r)
    Jacobian = jacsys(kzed,czed,r)
    Jentries = np.array([(Jacobian[0,0],Jacobian[0,1],Jacobian[1,0],Jacobian[1,1])])
    

    print("The No Commitment Case:")

print("The Jacobian Matrix of the No Committment case:")
print(Jacobian)
print(" ")
ew, ev = np.linalg.eig(Jacobian)
print('The Eigenvalues: {}'.format(ew))
print('The Eigenvectors as columns: {}'.format(np.dot(np.linalg.inv(ev),np.dot(Jacobian,ev))))

The No Commitment Case:
The Jacobian Matrix of the No Committment case:
[[ 0.02653    -1.        ]
 [-0.00165228  0.        ]]
 
The Eigenvalues: [ 0.05602293 -0.02949293]
The Eigenvectors as columns: [[ 5.60229294e-02  0.00000000e+00]
 [ 3.46944695e-18 -2.94929294e-02]]


In [20]:
t,v = rk4_solve(0,4,1e-2, UpLfQ,F,Jentries)

plt.plot(v[:,0],v[:,1], 'b');
       
plt.xlabel('k',size=15)
plt.ylabel('c',size=15);

IndexError: index 1 is out of bounds for axis 0 with size 1

In [18]:
UpLfQ = [-0.005, .005]  # Value in upper left quadrant (Blue)
UpRtQ = [0.01, .005]   # Value in upper right quadrant (Red)
LwLfQ = [-0.01, -0.005] # Value in lower left quadrantn(Purple)
LwRtQ = [0.005, -0.005]  # Value in lower right quadrant (Yellow)


t,v = rk4_solve(0,4,1e-2, UpLfQ,F,Jentries)
t2,v2 = rk4_solve(0,2,1e-2, UpRtQ,F,Jentries)
t3,v3 = rk4_solve(0,2,1e-2, LwLfQ,F,Jentries)
t4,v4 = rk4_solve(0,2,1e-2, LwRtQ,F,Jentries)

#np.shape(v)
plt.plot(v[:,0],v[:,1], 'b',v2[:,0], v2[:,1],'r',v3[:,0], v3[:,1],'m',v4[:,0], v4[:,1],'y',
         [-.009,.009],[0,0],'k',[0,0],[-.009, .009],'k');

plt.xlabel('k',size=15)
plt.ylabel('c',size=15);

IndexError: index 1 is out of bounds for axis 0 with size 1